In [1]:
import panel as pn
import altair as alt
import pandas as pd
import folium as fol
import datetime

from geopy.geocoders import Nominatim

from utils import filter_df

alt.renderers.enable('default')
pn.extension('vega')
alt.data_transformers.enable('csv')

DataTransformerRegistry.enable('csv')

In [2]:
import colorsys

def ColorFromNote(nota):
    if pd.isna(nota):
        return '#979797'
    rgb = colorsys.hsv_to_rgb(nota / 30., 1.0, 1.0)
    return '#%02x%02x%02x' % (int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

In [3]:
geolocator = Nominatim(user_agent="ReclameAqui")
location = geolocator.geocode("50740610, Brasil")
location

Location(Várzea, Recife, Região Geográfica Imediata do Recife, Região Metropolitana do Recife, Região Geográfica Intermediária do Recife, Pernambuco, Região Nordeste, 50740-610, Brasil, (-8.0406444, -34.9449161, 0.0))

In [6]:
df = pd.read_excel("dataset/LiveTim_maio2022.xlsx")

columns = list(df.columns)
df.head(5)


,Unnamed: 0,number,u_numero_protocolo,u_id_siebel,u_origem_ticket,opened_at,closed_at,state,u_service_id,u_ngnis_uf,...,u_cliente_autoriza_ori,u_velocidade_produto,u_wfm_cancelado_por,u_zip_ori,u_tipo_cliente_ori,u_ngnis_technology,location_lat,location_long,location_state,location_address
0,0,ATC00015506,2022327241010,1-JM6QFF3K,Siebel,2022-05-31 23:40:36,2022-06-01 15:40:07,Concluído,313076220,PE,...,Não,150Mbps/60Mbps,NaN,50740610,Residencial,GPON,-8.111562,-34.912657,Região Nordeste,"Acesso ao Banco do Brasil, Imbiribeira, Recife..."
1,1,ATC00015505,2022327160929,NaN,Whatsapp,2022-05-31 23:01:54,NaT,Iniciado,335092362,DF,...,Não,300Mbps/150Mbps,NaN,71070602,Residencial,GPON,-15.825883,-48.099923,Região Centro-Oeste,"QNM 5 Conjunto 0, Ceilândia Sul, Ceilândia, Re..."
2,2,ATC00015504,2022327137543,1-JM6SF6I5,Siebel,2022-05-31 22:35:36,2022-06-01 13:35:00,Concluído,184972613,AM,...,Não,150Mbps/40Mbps,NaN,69093770,Residencial,GPON,1.183321,-66.969221,Amazonas,"Brasil, Cucui, São Gabriel da Cachoeira, Regiã..."
3,3,ATC00015503,2022327051670,NaN,Whatsapp,2022-05-31 22:12:13,2022-06-01 10:25:00,Concluído,333877544,PE,...,Não,150Mbps/60Mbps,NaN,53435560,Residencial,GPON,-7.938184,-34.832967,Região Nordeste,"Rua Zero, Janga, Paulista, Região Geográfica I..."
4,4,ATC00015502,2022327064365,1-JM6P97KD,Siebel,2022-05-31 22:09:03,NaT,Cancelado,301675719,PE,...,Sim,200Mbps/100Mbps,NaN,50780100,Residencial,GPON,-8.111926,-34.912671,Região Nordeste,"Saida Banco do Brasil, Imbiribeira, Recife, Re..."


In [7]:
dfGroup = df.groupby(['location_lat', 'location_long']).agg({'number':'count'}).reset_index()


In [10]:
mapa = fol.Map(location=(-22.908333, -43.196388	), zoom_start=4, width='100%', height='100%',)

for index, row in dfGroup.iterrows():
   
    # customIcon = fol.Icon(color=ColorFromNote(row['Nota']))
    # if row['Nota'] < 6:
    #     customIcon = fol.Icon(color="red")
    # elif row['Nota'] < 8:
    #     customIcon = fol.Icon(color="beige")
    # elif row['Nota'] < 10:
    #     customIcon = fol.Icon(color="green")
    # else:
    #     customIcon = fol.Icon(color="gray", angle=30)

    # fol.Circle(
    #     location=(row['location_lat'],row['location_long']), 
    #     radius=size, 
    #     fill_color=ColorFromNote(row['Nota']), 
    #     fill_opacity=0.6, 
    #     color="black",
    #     tooltip=str(row['Nota']) +" - " +str(row['Id']),
    #     popup=str(row['Nota']) +" - " +str(row['Id']),
    #     weight=1).add_to(mapa)

    fol.Marker(
        location=(row['location_lat'],row['location_long']), 
        tooltip=str(row['number']),
        popup=str(row['number']),
        ).add_to(mapa)
    # try:
    #     fol.Marker(location=(row['location_lat'],row['location_long']), tooltip=str(row['Nota']) +" - " +str(row['Id']),icon=icon).add_to(mapa)
    # except ex:
    #     continue

mapa.save('LiveTimMap.html')
# display(mapa)